Mixing SQL queries into your python code may not be the ideal approach in every scenario, however sometimes an ORM just doesn't meet your complex querying needs and a SQL query is much simpler.

Regardless of the merits of using ORMs or not, this post assumes you are already in the situation where you have a complex SQL query sitting in your python application somewhere, and you want to test it just like you test your other functions. 
___


Let's begin by setting the scene and motivating the examples in this post.

We're currently working on a flask app and we've been asked to integrate a new data source to an existing view in the app. We take a look at the endpoint that renders the template for the view and this is what we're greeted with:

In [1]:
from flask import Flask
import sqlite3
from flask import g

DATABASE = 'database.db'

# for a real app we'd use the factory pattern
app = Flask(__name__)

# quickstart from https://flask.palletsprojects.com/en/2.2.x/patterns/sqlite3/
def get_db()-> sqlite3.Connection:
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

@app.get('/')
def home():
    query = "SELECT * FROM table1"
    cur = get_db().execute(query)
    res = cur.fetchall()
    return res

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Nov/2022 21:19:59] "GET / HTTP/1.1" 200 -
